# Process the Ocean Carbonate data 
'OceanSODA-ETHZ: A global gridded data set of the surface
ocean carbonate system for seasonal to decadal studies of
ocean acidification'

*Methodology* doc: https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0220059/OceanSODA-ETHZ_v2021_README.pdf

*Requirements*: need netcdf4 backend on geopandas to open net-cdf - use 'conda install -c conda-forge netcdf4'

*Data*: Can download the nc file at  https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0220059/

In [1]:
import pandas as pd
import glob
import datetime
from shapely.geometry import Point
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import PyCO2SYS as pyco2

In [18]:
#Define functions needed

def weighted_temporal_mean_multi(ds, list_var):
    """
    Roll up nc file to yearly averages: Weight by days in each month to create yearly averages accuratly 
    Filter to rough bounding lat/lon box of the USA
    
    Inputs:
    ds: xarray dataset of nc file 
    list_var: list of the variables to be included in the final output (numeric only) 
    """
    # Determine the month length
    month_length = ds.time.dt.days_in_month

    # Calculate the weights
    wgts = month_length.groupby("time.year") / month_length.groupby("time.year").sum()

    # Make sure the weights in each year add up to 1
    np.testing.assert_allclose(wgts.groupby("time.year").sum(xr.ALL_DIMS), 1.0)

    # Subset our dataset for our variable
    obs = ds[list_var]

    # Setup our masking for nan values
    cond = obs.isnull()
    ones = xr.where(cond, 0.0, 1.0)

    # Calculate the numerator
    obs_sum = (obs * wgts).resample(time="AS").sum(dim="time")

    # Calculate the denominator
    ones_out = (ones * wgts).resample(time="AS").sum(dim="time")
    # Filter to USA bounds 
    
    df = obs_sum / ones_out
    df = df.sel( lon=slice(-129, -60), lat=slice(22, 60))
    # Return the weighted average
    return df

def process_df_DRI(df):
    """
    Process the yearly average df to add the dDICdTA etc. 
    """
    #Add the point geoemtry and extract the Year
    df['geometry'] = [Point(x,y) for x,y in zip( df['lon'], df['lat']) ]
    df['Y'] = [x.year for x in df.time]
    gdf = gpd.GeoDataFrame(df, geometry = 'geometry', crs ='EPSG:4326')
    
    #Create the DRI
    for i, row in gdf.iterrows():
        ph = row['ph_total'] #pH in total scale -  default
        T = row['temperature'] # degC
        TA = row['talk'] #umol per kh
        salin=  row['salinity'] # ppt

        # Define the known marine carbonate system parameters
        par1 = TA  # Total Alkalinity (meq/L or μmol/kg)
        par2 = ph  # pH in total scale ; default 
        par1_type = 1  # tell PyCO2SYS: "par1 is a TA value"
        par2_type = 3  # tell PyCO2SYS: "par2 is a pH value"

        # Set up a dict for the keyword arguments, for convenience
        pyco2_kws = {}

        # Define the seawater conditions and add them to the dict
        # NB salinity set to zero in freshwater case
        pyco2_kws["temperature"] = T  # lab temperature (input conditions) in °C
        pyco2_kws["pressure"] = 0  # lab pressure (input conditions) in dbar, ignoring the atmosphere
        pyco2_kws["salinity"] =  salin

        pyco2_kws["opt_k_carbonic"] = 10  # tell PyCO2SYS: Total pH scale, real seawater).

        # Now calculate everything with PyCO2SYS!
        results = pyco2.sys(par1, par2, par1_type, par2_type, **pyco2_kws)

        # pH_model = results['pH']
        DIC_model = results['dic'] # dic = CO2 + HCO2 + CO3
        HCO3_model = results['HCO3']
        CO3_model = results['CO3']
        CO2_model = results['CO2']

        # Which parameter(s) do we want to propagate uncertainties into?
        uncertainties_into = ["dic",'HCO3','CO3','CO2']

        # Define measurement uncertainties
        uncertainties_from = {
            "par1": 1, # uncertainty in TA, 1meq/L
        }

        # Propagate and print out results
        uncertainties, components = pyco2.uncertainty.propagate_nd(
            results, uncertainties_into, uncertainties_from, **pyco2_kws
        )

        dDICdTA = uncertainties['dic']
        dHCO3dTA = uncertainties['HCO3']
        dCO3dTA = uncertainties['CO3']
        dCO2dTA = uncertainties['CO2']

        gdf.loc[i, 'dDICdTA'] = dDICdTA
        gdf.loc[i, 'dHCO3dTA'] = dHCO3dTA
        gdf.loc[i, 'dCO3dTA'] = dCO3dTA
        gdf.loc[i, 'dCO2dTA'] = dCO2dTA
    
    return gdf

def test_pyco3_one_sample(df):
    ph = df['ph_total'] #pH in total scale -  default
    T = df['temperature'] # degC
    TA = df['talk'] #umol per kh
    salin=  df['salinity'] # ppt

    # Define the known marine carbonate system parameters
    par1 = TA  # Total Alkalinity (meq/L or μmol/kg)
    par2 = ph  # pH in total scale ; default 
    par1_type = 1  # tell PyCO2SYS: "par1 is a TA value"
    par2_type = 3  # tell PyCO2SYS: "par2 is a pH value"

    # Set up a dict for the keyword arguments, for convenience
    pyco2_kws = {}

    # Define the seawater conditions and add them to the dict
    # NB salinity set to zero in freshwater case
    pyco2_kws["temperature"] = T  # lab temperature (input conditions) in °C
    pyco2_kws["pressure"] = 0  # lab pressure (input conditions) in dbar, ignoring the atmosphere
    pyco2_kws["salinity"] =  salin

    pyco2_kws["opt_k_carbonic"] = 10  # tell PyCO2SYS: Total pH scale, real seawater).

    # Now calculate everything with PyCO2SYS!
    results = pyco2.sys(par1, par2, par1_type, par2_type, **pyco2_kws)
    return results
    

# Process data

In [3]:
fn = '/Users/gracecolverd/EionTeam/Eion/river_app/ocean_co_data/DAA/OceanSODA-ETHZ_GRaCER_v2021a_1982-2020.nc'
xdr = xr.open_dataset(fn)
xdr

<xarray.Dataset>
Dimensions:          (time: 468, lat: 180, lon: 360)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) datetime64[ns] 1982-01-01 1982-02-01 ... 2020-12-01
Data variables: (12/23)
    talk             (time, lat, lon) float32 ...
    dic              (time, lat, lon) float32 ...
    spco2            (time, lat, lon) float32 ...
    sfco2            (time, lat, lon) float32 ...
    ph_total         (time, lat, lon) float32 ...
    hco3             (time, lat, lon) float32 ...
    ...               ...
    salinity         (time, lat, lon) float32 ...
    talk_uncert      (time, lat, lon) float32 ...
    sfco2_uncert     (time, lat, lon) float32 ...
    fgco2            (time, lat, lon) float32 ...
    fgco2_global     (time) float32 ...
    area             (lat, lon) float32 ...
Attributes:
    contact:      gregorl@ethz.ch
    author:       Luke Gregor
    institution:  ETH Zuerich
    version:      v2021e
    date:         2021-08-09
    description:  talk and pco2 (more accurately fco2) are estimated with two...
    changelog:    v2021d: Extended from 1982-2020; Now using: OISSTv2.1 for S...
    reference:    Gregor, L. and Gruber, N.: OceanSODA-ETHZ: A global gridded...
    source:       https://doi.org/10.25921/m5wx-ja34
    product:      OceanSODA-ETHZ-v2021e

In [4]:
#Create yearly average for chosen variables 
list_vars = ['temperature','talk', 'dic', 'ph_total', 'spco2', 'hco3', 'co3', 'co2', 'sfco2','fgco2', 'dic_uncert', 'spco2_uncert', 'ph_total_uncert','salinity', 'talk_uncert' ] 

multi = weighted_temporal_mean_multi(xdr , list_vars) 

In [5]:
#Convert to de-indexed dataframe 
df = multi.to_dataframe().dropna().reset_index()

In [6]:
df.head()

,time,lat,lon,temperature,talk,dic,ph_total,spco2,hco3,co3,co2,sfco2,fgco2,dic_uncert,spco2_uncert,ph_total_uncert,salinity,talk_uncert
0,1982-01-01,22.5,-128.5,21.901815,2268.947606,1965.374550,8.114171,327.758885,1741.485103,213.795473,10.093966,326.673184,-0.220523,5.689380,4.889975,0.005330,34.224556,5.830043
1,1982-01-01,22.5,-127.5,21.799232,2268.265082,1967.379590,8.111411,330.346272,1745.169477,212.008025,10.202077,329.250666,-0.126980,5.626352,4.805515,0.005206,34.211570,5.817049
2,1982-01-01,22.5,-126.5,21.670198,2267.398499,1968.472419,8.109904,331.665888,1747.560789,210.634330,10.277318,330.564184,-0.077676,5.685884,4.972467,0.005360,34.228341,5.815351
3,1982-01-01,22.5,-125.5,21.642712,2265.279570,1967.709735,8.108464,332.758997,1747.709044,209.681971,10.318719,331.653314,-0.039491,6.166006,5.044610,0.005457,34.214447,6.419736
4,1982-01-01,22.5,-124.5,21.665129,2263.468294,1967.783374,8.104790,335.887412,1749.007469,208.366411,10.409484,334.771622,0.061367,6.136014,5.299114,0.005676,34.239235,6.290526


In [11]:
#Turn df into geo-df and add DRI 
gdf = process_df_DRI(df)

/Users/gracecolverd/miniconda3/envs/ncfiles/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [46]:
gdf.head()


,time,lat,lon,temperature,talk,dic,ph_total,spco2,hco3,co3,...,spco2_uncert,ph_total_uncert,salinity,talk_uncert,geometry,Y,dDICdTA,dHCO3dTA,dCO3dTA,dCO2dTA
0,1982-01-01 00:00:00,22.5,-128.5,21.901815,2268.947606,1965.374550,8.114171,327.758885,1741.485103,213.795473,...,4.889975,0.005330,34.224556,5.830043,POINT (-128.50000 22.50000),1982,0.905783,0.802276,0.098862,0.004645
1,1982-01-01 00:00:00,22.5,-127.5,21.799232,2268.265082,1967.379590,8.111411,330.346272,1745.169477,212.008025,...,4.805515,0.005206,34.211570,5.817049,POINT (-127.50000 22.50000),1982,0.906661,0.803932,0.098034,0.004695
2,1982-01-01 00:00:00,22.5,-126.5,21.670198,2267.398499,1968.472419,8.109904,331.665888,1747.560789,210.634330,...,4.972467,0.005360,34.228341,5.815351,POINT (-126.50000 22.50000),1982,0.907316,0.805172,0.097414,0.004730
3,1982-01-01 00:00:00,22.5,-125.5,21.642712,2265.279570,1967.709735,8.108464,332.758997,1747.709044,209.681971,...,5.044610,0.005457,34.214447,6.419736,POINT (-125.50000 22.50000),1982,0.907705,0.805903,0.097049,0.004753
4,1982-01-01 00:00:00,22.5,-124.5,21.665129,2263.468294,1967.783374,8.104790,335.887412,1749.007469,208.366411,...,5.299114,0.005676,34.239235,6.290526,POINT (-124.50000 22.50000),1982,0.908298,0.807002,0.096499,0.004797


# Validations

In [33]:
#Validations: spot check that pyco2 output matches the grid / predictions results from the original data 
for x in (0,100,1000):
    grid_one_sample = gdf.iloc[x]

    results = test_pyco3_one_sample(  grid_one_sample )
    print('Sample {}'.format(x) )
    print('DIC Grid vs Pyco2 % diff: {} '.format((( grid_one_sample['dic'] - results['dic'] ) /grid_one_sample['dic'] * 100).round(3) ) )
    print('HCO3 Grid vs Pyco2 % diff: {} '.format((( grid_one_sample['hco3'] - results['HCO3'] ) /grid_one_sample['hco3'] * 100).round(3) ) )
    print('CO3 Grid vs Pyco2 % diff: {} '.format((( grid_one_sample['co3'] - results['CO3'] ) /grid_one_sample['co3'] * 100).round(3) ) )
    print('CO2 Grid vs Pyco2 % diff: {} '.format((( grid_one_sample['co2'] - results['CO2'] ) /grid_one_sample['co2'] * 100).round(3) ) )

Sample 0
DIC Grid vs Pyco2 % diff: 0.016 
HCO3 Grid vs Pyco2 % diff: 0.056 
CO3 Grid vs Pyco2 % diff: -0.319 
CO2 Grid vs Pyco2 % diff: 0.17 
Sample 100
DIC Grid vs Pyco2 % diff: 0.1 
HCO3 Grid vs Pyco2 % diff: 0.227 
CO3 Grid vs Pyco2 % diff: -0.759 
CO2 Grid vs Pyco2 % diff: 1.326 
Sample 1000
DIC Grid vs Pyco2 % diff: 0.007 
HCO3 Grid vs Pyco2 % diff: 0.042 
CO3 Grid vs Pyco2 % diff: -0.33 
CO2 Grid vs Pyco2 % diff: 0.159 


In [16]:
# Check the output ranges of dDICdTA: min, max and mean 
gdf.dDICdTA.min(),  gdf.dDICdTA.max(), gdf.dDICdTA.mean() 

(0.8843961836646486, 0.9722952131596685, 0.9145945210757103)

In [15]:
#Min and max pH
gdf.ph_total.min(), gdf.ph_total.max()

(8.007416649594335, 8.287404363449307)

# Export Processed data  

In [37]:
gdf['time'] = [str(x) for x in gdf.time ]          
 

In [43]:
import os

In [45]:
gdf.to_file(os.getcwd() + '/data/ocean_grid/ocean_carbonate_grid_data.shp') 

/var/folders/n5/_2d3c5sd5j3810327mknn0sh0000gn/T/ipykernel_28232/1124727752.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(os.getcwd() + '/data/ocean_grid/ocean_carbonate_grid_data.shp')
